# Titanic using PyCaret

Import PyCaret and Pandas, and import the titanic dataset into a dataframe. Show the top couple of rows.

In [26]:
#DELETE

import pandas as pd
from pycaret.classification import *

df_with = pd.read_excel('files/titanic3.xlsx', engine='openpyxl')
df_with.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


Remember how having boat and body columns was cheating? Create 2 datasets, one containg boat and body and another one without. This will allow us to compare the results for both.

In [27]:
#DELETE
df_without = df_with.drop(['boat', 'body'], axis=1)

Because we are building two models we won't be able to use the functional api bet we'll have the use the OOP-way. Also make sure to set the session_id to direct the randomness. Lets take 42.

In [43]:
#DELETE
model_with = ClassificationExperiment()
model_without = ClassificationExperiment()

model_with.setup(data=df_with, target='survived', session_id=42)
model_without.setup(data=df_without, target='survived', session_id=42)

,Description,Value
0,Session id,42
1,Target,survived
2,Target type,Binary
3,Original data shape,"(1309, 14)"
4,Transformed data shape,"(1309, 16)"
5,Transformed train set shape,"(916, 16)"
6,Transformed test set shape,"(393, 16)"
7,Numeric features,6
8,Categorical features,7
9,Rows with missing values,100.0%


,Description,Value
0,Session id,42
1,Target,survived
2,Target type,Binary
3,Original data shape,"(1309, 12)"
4,Transformed data shape,"(1309, 14)"
5,Transformed train set shape,"(916, 14)"
6,Transformed test set shape,"(393, 14)"
7,Numeric features,5
8,Categorical features,6
9,Rows with missing values,81.7%


Since training takes a bit longer, let's do this in two separate code cells.

In [44]:
#DELETE
best_with = model_with.compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:45:19
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9519,0.9847,0.8943,0.9777,0.9330,0.8958,0.8991,0.5270
ridge,Ridge Classifier,0.9519,0.9823,0.8857,0.9877,0.9326,0.8955,0.9002,0.0280
et,Extra Trees Classifier,0.8481,0.0000,0.6743,0.9064,0.7610,0.6576,0.6800,0.0590
knn,K Neighbors Classifier,0.7269,0.0000,0.6000,0.6555,0.6245,0.4113,0.4138,0.3340
rf,Random Forest Classifier,0.6495,0.0000,0.0829,0.8000,0.1442,0.0988,0.1970,0.0660
svm,SVM - Linear Kernel,0.6211,0.8292,0.3971,0.4746,0.3518,0.1658,0.1898,0.0260
nb,Naive Bayes,0.6179,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0260
dt,Decision Tree Classifier,0.6179,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0340
ada,Ada Boost Classifier,0.6179,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0290
gbc,Gradient Boosting Classifier,0.6179,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0410


In [45]:
#DELETE
best_without = model_without.compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:45:48
Status,. . . . . . . . . . . . . . . . . .,Initializing CV
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7870,0.8496,0.6029,0.7925,0.6819,0.5270,0.5401,0.0470
ridge,Ridge Classifier,0.7106,0.8131,0.3571,0.7677,0.4815,0.3177,0.3649,0.0260
et,Extra Trees Classifier,0.7073,0.0000,0.3571,0.7618,0.4774,0.3110,0.3574,0.0550
knn,K Neighbors Classifier,0.6767,0.0000,0.4857,0.5948,0.5318,0.2899,0.2948,0.0330
svm,SVM - Linear Kernel,0.6408,0.6724,0.5571,0.5184,0.4875,0.2381,0.2677,0.0240
nb,Naive Bayes,0.6179,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0250
dt,Decision Tree Classifier,0.6179,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0280
rf,Random Forest Classifier,0.6179,0.0000,0.0200,0.3067,0.0369,0.0093,0.0295,0.0680
ada,Ada Boost Classifier,0.6179,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0250
gbc,Gradient Boosting Classifier,0.6179,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0410


As expected the model without boat and body is worse (accuracy of 0.79 vs 0.95) than the model with. But it's still a good model, better than our own model (which had accuracy 0.77). But the main difference is the amount of work we had to put into it: No splitting data, dropping Na-values, ...

All this is done automatically by PyCaret in the background. What PyCaret also did is feature engineering (filling in Na-values, ...). If we wish we could delve into that and imporve the model, which would be a good idea when you plan on using it in production. We however have a whole new chapter ready for that sort of stuff.

# Using the model

What we will be doing is using the models we created to do a prediction.

In [46]:
random_person_dict = {
    'pclass': 1,
    # 'survived': 0,
    'name': 'John Doe',
    'sex': 'male',
    'age': 35,
    'sibsp': 3, # SibSp is the number of siblings or spouse of a person onboard.
    'parch': 2, # Similar to the SibSp, this feature contained the number of parents or children each passenger was touring with.    
    'ticket': '12586641',
    'fare': df.fare.mean(),
    'cabin': 'C35',
    'embarked': 'S',
    'home.dest': 'New York, NY'
}

random_person = pd.DataFrame(random_person_dict, index=[0])

random_person

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,1,John Doe,male,35,3,2,12586641,33.295479,C35,S,"New York, NY"


With this random person, use the model without boat and body to predict survival.

In [47]:
prediction_without = model_without.predict_model(best_without, data=random_person)
prediction_without

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest,prediction_label,prediction_score
0,1,John Doe,male,35,3,2,12586641,33.295479,C35,S,"New York, NY",0,0.8593


Maybe John Doe better do some [shopping](https://www.decathlon.be/nl/alle-sporten/kajakken/kano-kajak) before leaving.

## Cleanup

PyCaret creates a couple of files in the folder where it is run. You should see "logs.log" and "my_best_pipeline.pkl". You can delete these before moving on.